In [1]:
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
df = pd.read_csv(r"D:\ProjectDeployed_flight_prediction\data\Clean_Dataset.csv")

In [4]:
df.head()

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [5]:
df['airline'].value_counts()

airline
Vistara      127859
Air_India     80892
Indigo        43120
GO_FIRST      23173
AirAsia       16098
SpiceJet       9011
Name: count, dtype: int64

In [6]:
x = df[['airline', 'source_city','departure_time', 'stops','arrival_time', "class", 'destination_city', 'duration', 'days_left']]
y = df['price']

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size =.3, random_state = 360)

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler

# drop weak variables 
x_train.drop(["days_left","duration"], axis =1, inplace =True)
x_test.drop(["days_left","duration"], axis =1, inplace =True)

# create function to transform variables
ord_mapping =[["Early_Morning","Morning","Afternoon", "Evening","Night","Late_Night" ],
              ["Early_Morning","Morning","Afternoon", "Evening","Night","Late_Night" ], 
              ["zero","one","two_or_more" ]]

t= ColumnTransformer(transformers=[
    ("onehot", OneHotEncoder(),["airline", "source_city", "destination_city", "class"]),
    ('ordinal', OrdinalEncoder(categories=ord_mapping), ["departure_time", "arrival_time", "stops"])])


# transform variables 
Train_features  = t.fit_transform(x_train)
test_features = t.fit_transform(x_test)

In [8]:
# run cross validation
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LinearRegression
cross_val = cross_val_score(LinearRegression(), Train_features,y_train, cv=10)

print('Cross-Validation Accuracy Scores -\n', cross_val, "\n\n")
print('Cross-Validation Accuracy Scores AVG -', cross_val.mean())

Cross-Validation Accuracy Scores -
 [0.90223343 0.90269147 0.90288702 0.90224496 0.90239874 0.90314608
 0.902932   0.90436976 0.90395985 0.90696599] 


Cross-Validation Accuracy Scores AVG - 0.9033829293750053


In [9]:
linear_regressor=LinearRegression()

In [10]:
linear_regressor.fit(Train_features,y_train)

LinearRegression()

In [11]:
cross_val = cross_val_score(LinearRegression(), test_features,y_test, cv=10)

print('Cross-Validation Accuracy Scores -\n', cross_val, "\n\n")
print('Cross-Validation Accuracy Scores AVG -', cross_val.mean())

Cross-Validation Accuracy Scores -
 [0.90291832 0.90382708 0.90715    0.90411917 0.90186898 0.90057364
 0.90149702 0.89850213 0.90461437 0.90210186] 


Cross-Validation Accuracy Scores AVG - 0.9027172566472604


In [12]:
import pickle

In [13]:
pickle.dump(t,open(r"transformer.pkl",'wb'))
pickle.dump(linear_regressor,open(r"model.pkl",'wb'))

In [14]:
df['stops'].value_counts()

stops
one            250863
zero            36004
two_or_more     13286
Name: count, dtype: int64

In [15]:
df['departure_time'].value_counts()

departure_time
Morning          71146
Early_Morning    66790
Evening          65102
Night            48015
Afternoon        47794
Late_Night        1306
Name: count, dtype: int64

In [16]:
df['source_city'].value_counts()

source_city
Delhi        61343
Mumbai       60896
Bangalore    52061
Kolkata      46347
Hyderabad    40806
Chennai      38700
Name: count, dtype: int64

In [17]:
df['destination_city'].value_counts()

destination_city
Mumbai       59097
Delhi        57360
Bangalore    51068
Kolkata      49534
Hyderabad    42726
Chennai      40368
Name: count, dtype: int64